In [1]:
from datetime import datetime, timedelta
from google.cloud import storage
import ee

import pretty_errors  # for pretty printing of error messages

In [3]:
from dotenv import load_dotenv
import os

# Load environment variables from a .env file located in the same directory as this script
load_dotenv()

# Now you can access the environment variable
cloud_project = os.environ.get("GOOGLE_CLOUD_PROJECT_NAME")
key_path = os.environ.get("GOOGLE_CLOUD_KEY_PATH")

In [4]:
ee.Authenticate()
ee.Initialize(project=cloud_project)

In [5]:
bbox = ee.Geometry.Polygon( # all of costa rica
    [[
        [-85.9, 8.0],  # Lower left corner (southwest)
        [-85.9, 11.2], # Upper left corner (northwest)
        [-82.5, 11.2], # Upper right corner (northeast)
        [-82.5, 8.0],  # Lower right corner (southeast)
        [-85.9, 8.0]   # Closing the polygon by repeating the first point
    ]]
)

In [19]:
# Convert the dates to datetime objects
start_date = datetime.strptime('2018-10-02', '%Y-%m-%d')
end_date = datetime.strptime('2018-10-02', '%Y-%m-%d')

# Calculate the new dates
before_start = (start_date - timedelta(days=10)).strftime("%Y-%m-%d")
before_end = start_date.strftime("%Y-%m-%d")

after_start = end_date.strftime("%Y-%m-%d")
after_end = (end_date + timedelta(days=10)).strftime("%Y-%m-%d")

print(f"Generating training data for {start_date} to {end_date}...")

year_before_start = start_date - timedelta(days=365)
start_of_year = datetime(year_before_start.year, 1, 1)
end_of_year = datetime(year_before_start.year, 12, 31)

# Load the datasets

dem = ee.Image("WWF/HydroSHEDS/03VFDEM").clip(bbox)
slope = ee.Terrain.slope(dem)
landcover = ee.Image("ESA/WorldCover/v100/2020").select("Map").clip(bbox)
flow_direction = ee.Image("WWF/HydroSHEDS/03DIR").clip(bbox)

stream_dist_proximity_collection = (
    ee.ImageCollection(
        "projects/sat-io/open-datasets/HYDROGRAPHY90/stream-outlet-distance/stream_dist_proximity"
    )
    .filterBounds(bbox)
    .mosaic()
)
stream_dist_proximity = stream_dist_proximity_collection.clip(bbox).rename(
    "stream_distance"
)

flow_accumulation_collection = (
    ee.ImageCollection(
        "projects/sat-io/open-datasets/HYDROGRAPHY90/base-network-layers/flow_accumulation"
    )
    .filterBounds(bbox)
    .mosaic()
)
flow_accumulation = flow_accumulation_collection.clip(bbox).rename(
    "flow_accumulation"
)

spi_collection = (
    ee.ImageCollection("projects/sat-io/open-datasets/HYDROGRAPHY90/flow_index/spi")
    .filterBounds(bbox)
    .mosaic()
)
spi = spi_collection.clip(bbox).rename("spi")

sti_collection = (
    ee.ImageCollection("projects/sat-io/open-datasets/HYDROGRAPHY90/flow_index/sti")
    .filterBounds(bbox)
    .mosaic()
)
sti = sti_collection.clip(bbox).rename("sti")

cti_collection = (
    ee.ImageCollection("projects/sat-io/open-datasets/HYDROGRAPHY90/flow_index/cti")
    .filterBounds(bbox)
    .mosaic()
)
cti = cti_collection.clip(bbox).rename("cti")

tpi_collection = (
    ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/tpi")
    .filterBounds(bbox)
    .mosaic()
)
tpi = tpi_collection.clip(bbox).rename("tpi")

tri_collection = (
    ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/tri")
    .filterBounds(bbox)
    .mosaic()
)
tri = tri_collection.clip(bbox).rename("tri")

pcurv_collection = (
    ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/pcurv")
    .filterBounds(bbox)
    .mosaic()
)
pcurv = pcurv_collection.clip(bbox).rename("pcurv")

tcurv_collection = (
    ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/tcurv")
    .filterBounds(bbox)
    .mosaic()
)
tcurv = tcurv_collection.clip(bbox).rename("tcurv")

aspect_collection = (
    ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/aspect")
    .filterBounds(bbox)
    .mosaic()
)
aspect = aspect_collection.clip(bbox).rename("aspect")

# SET SAR PARAMETERS (can be left default)

# Polarization (choose either "VH" or "VV")
polarization = "VH"  # or "VV"

# Pass direction (choose either "DESCENDING" or "ASCENDING")
pass_direction = "DESCENDING"  # or "ASCENDING"

# Difference threshold to be applied on the difference image (after flood - before flood)
# It has been chosen by trial and error. Adjust as needed.
difference_threshold = 1.25

# Relative orbit (optional, if you know the relative orbit for your study area)
# relative_orbit = 79

# Load and filter Sentinel-1 GRD data by predefined parameters
collection = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filter(ee.Filter.eq("instrumentMode", "IW"))
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", polarization))
    .filter(ee.Filter.eq("orbitProperties_pass", pass_direction))
    .filter(ee.Filter.eq("resolution_meters", 10))
    .filterBounds(bbox)
    .select(polarization)
)

# Select images by predefined dates
before_collection = collection.filterDate(before_start, before_end)
after_collection = collection.filterDate(after_start, after_end)

# Create a mosaic of selected tiles and clip to the study area
before = before_collection.mosaic().clip(bbox)
after = after_collection.mosaic().clip(bbox)

# Apply radar speckle reduction by smoothing
smoothing_radius = 50
before_filtered = before.focal_mean(smoothing_radius, "circle", "meters")
after_filtered = after.focal_mean(smoothing_radius, "circle", "meters")

# Calculate the difference between the before and after images
difference = after_filtered.divide(before_filtered)

# Apply the predefined difference-threshold and create the flood extent mask
threshold = difference_threshold
difference_binary = difference.gt(threshold)

# Refine the flood result using additional datasets
swater = ee.Image("JRC/GSW1_0/GlobalSurfaceWater").select("seasonality")
swater_mask = swater.gte(10).updateMask(swater.gte(10))
flooded_mask = difference_binary.where(swater_mask, 0)
flooded = flooded_mask.updateMask(flooded_mask)
connections = flooded.connectedPixelCount()
flooded = flooded.updateMask(connections.gte(8))

# Mask out areas with more than 5 percent slope using a Digital Elevation Model
flooded = flooded.updateMask(slope.lt(5))

hydro_proj = stream_dist_proximity.projection()

# Set the default projection from the hydrography dataset
flooded = flooded.setDefaultProjection(hydro_proj)

# Create a full-area mask, initially marking everything as non-flooded (value 0)
full_area_mask = ee.Image.constant(0).clip(bbox)

# Update the mask to mark flooded areas (value 1)
# Assuming flooded_mode is a binary image with 1 for flooded areas and 0 elsewhere
flood_labeled_image = full_area_mask.where(flooded, 1)

# Now flood_labeled_image contains 1 for flooded areas and 0 for non-flooded areas

combined = (
    dem.rename("elevation")
    .addBands(landcover.select("Map").rename("landcover"))
    .addBands(slope)
    .addBands(flow_direction.rename("flow_direction"))
    .addBands(stream_dist_proximity)
    .addBands(flow_accumulation)
    .addBands(spi)
    .addBands(sti)
    .addBands(cti)
    .addBands(tpi)
    .addBands(tri)
    .addBands(pcurv)
    .addBands(tcurv)
    .addBands(aspect)
    .addBands(flood_labeled_image.rename("flooded_mask"))
)


Generating training data for 2018-10-02 00:00:00 to 2018-10-02 00:00:00...


In [20]:
# return the start time of the combined image
start_time = combined.get("system:time_start").getInfo()
print(start_time)

None


In [ ]:
# hydrosheds dem min max: -440* 	8527* 


In [9]:
import restee as ree

# get an authenticated cloud session for requesting data
session = ree.EESession(cloud_project, key_path)

In [13]:
cr_domain = ree.Domain.from_ee_geometry(session,bbox,resolution=0.005)

In [14]:
data_xr = ree.img_to_xarray(session,cr_domain,combined,no_data_value='NaN')

In [15]:
data_xr

<xarray.Dataset> Size: 33MB
Dimensions:            (lat: 644, lon: 682)
Coordinates:
  * lon                (lon) float64 5kB -85.9 -85.9 -85.89 ... -82.5 -82.5
  * lat                (lat) float64 5kB 11.21 11.2 11.2 ... 8.002 7.997 7.992
Data variables: (12/15)
    elevation          (lat, lon) float32 2MB nan -9.999e+03 ... nan nan
    landcover          (lat, lon) float32 2MB nan 80.0 80.0 80.0 ... nan nan nan
    slope              (lat, lon) float32 2MB nan 0.0 0.0 0.0 ... nan nan nan
    flow_direction     (lat, lon) float32 2MB nan 247.0 247.0 ... nan nan nan
    stream_distance    (lat, lon) float64 4MB nan 0.0 0.0 0.0 ... nan nan nan
    flow_accumulation  (lat, lon) float32 2MB nan 0.0 0.0 0.0 ... nan nan nan
    ...                 ...
    tpi                (lat, lon) float32 2MB nan 0.0 0.0 0.0 ... nan nan nan
    tri                (lat, lon) float32 2MB nan 0.0 0.0 0.0 ... nan nan nan
    pcurv              (lat, lon) float32 2MB nan 0.0 0.0 0.0 ... nan nan nan
    tcurv              (lat, lon) float32 2MB nan 0.0 0.0 0.0 ... nan nan nan
    aspect             (lat, lon) float32 2MB nan 0.0 0.0 0.0 ... nan nan nan
    flooded_mask       (lat, lon) float32 2MB nan 0.0 0.0 0.0 ... nan nan nan